<a href="https://colab.research.google.com/github/AyazMurtazin/PythonLibsPractice/blob/main/%D0%91%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%B0_ibis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Библиотека Ibis



## Введение



Ibis — это библиотека Python, предназначенная для работы с большими объемами данных. Она предоставляет простой и интуитивно понятный интерфейс для создания сложных запросов и анализа данных, позволяя пользователям легко интегрировать работу с базами данных, такими как SQL, с анализом данных в Python. Библиотека Ibis позволяет писать код на Python, который затем преобразуется в SQL, что делает её удобной для аналитиков и разработчиков, которые предпочитают работать с высокоуровневыми языками, а не с сырыми SQL-запросами.



### Основные возможности Ibis

- **Абстракция источников данных**: Ibis поддерживает различные источники данных, такие как SQL-базы данных, Parquet-файлы, данные в Hadoop и многие другие.
- **Создание выражений**: Ibis позволяет создавать сложные запросы с помощью выражений, которые затем преобразуются в SQL.
- **Интеграция с Pandas**: Позволяет работать с данными в стиле Pandas, что делает библиотеку удобной для пользователей, знакомых с этой библиотекой.
- **Параллельная обработка**: Возможности параллельной обработки позволяют эффективно работать с большими наборами данных.



## Установка Ibis

Для начала работы с библиотекой Ibis необходимо установить её. Это можно сделать с помощью pip:




In [ ]:
!pip install ibis-framework




## Основные концепции Ibis



### 1. Ibis Expressions

Ibis использует концепцию выражений, которая позволяет пользователям описывать операции с данными, не указывая конкретные SQL-запросы. Например, мы можем описать выражение, которое будет производить агрегацию данных.

### 2. Таблицы и столбцы

В Ibis данные организованы в виде таблиц и столбцов, что делает её схожей с реляционными базами данных. Каждая таблица представляется объектом, который позволяет нам взаимодействовать со столбцами и выполнять операции, такие как фильтрация, агрегация и группировка.

## Пример работы с Ibis

Теперь рассмотрим простой пример работы с Ibis. Для этого мы будем использовать SQLite как источник данных, но Ibis поддерживает множество других источников, таких как PostgreSQL, MySQL и другие.

### Создание базы данных SQLite

Для демонстрации работы с Ibis создадим простую SQLite базу данных с таблицей "employees".




In [ ]:
import sqlite3



In [ ]:
# Создаем соединение с базой данных

connection = sqlite3.connect('example.db')


# Создаем курсор для выполнения SQL-запросов

cursor = connection.cursor()


# Создаем таблицу employees

cursor.execute('''
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT,
    department TEXT,
    salary REAL
)
''')


# Добавляем данные

employees = [
    (1, 'Alice', 'HR', 70000),
    (2, 'Bob', 'Engineering', 80000),
    (3, 'Charlie', 'Sales', 60000),
    (4, 'David', 'Engineering', 90000),
    (5, 'Eva', 'HR', 75000)
]

cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?)', employees)


# Сохраняем изменения и закрываем соединение

connection.commit()
connection.close()




В этом примере мы создали таблицу `employees`, содержащую информацию о сотрудниках, включая их имя, отдел и зарплату.



### Подключение Ibis к базе данных



Теперь давайте подключим Ibis к нашей базе данных и создадим выражение для выборки данных.




In [ ]:
import ibis

# Подключаемся к базе данных SQLite
ibis_connection = ibis.sqlite.connect('example.db')

# Получаем таблицу employees
employees_table = ibis_connection.table('employees')




### Создание простого запроса

Теперь, когда у нас есть таблица, давайте создадим простой запрос, который выберет всех сотрудников с зарплатой выше 70000.



In [ ]:
# Создаем выражение для фильтрации
query = employees_table[employees_table.salary > 70000]

# Преобразуем выражение в SQL
sql_query = query.compile()

# Выводим SQL-запрос
print(sql_query)




SELECT * FROM "employees" AS "t0" WHERE "t0"."salary" > 70000


#### Объяснение кода

- Мы используем `employees_table[salary > 70000]` для создания выражения, которое выбирает всех сотрудников с зарплатой выше 70000.
- Метод `compile()` преобразует выражение Ibis в SQL-запрос, который можно выполнить в базе данных.

### Выполнение запроса

Для выполнения запроса мы можем использовать метод `execute()`.




In [ ]:
# Выполняем запрос и получаем результат
result = query.execute()

# Выводим результат
print(result)




   id   name   department   salary
0   2    Bob  Engineering  80000.0
1   4  David  Engineering  90000.0
2   5    Eva           HR  75000.0


### Пример агрегации данных

Теперь рассмотрим пример агрегации данных. Мы можем посчитать среднюю зарплату сотрудников по отделам.




In [ ]:
# Создаем выражение для группировки и агрегации
average_salary = (
    employees_table.group_by('department')
    .aggregate(avg_salary=employees_table.salary.mean())
)

# Преобразуем выражение в SQL
sql_query_avg = average_salary.compile()

# Выводим SQL-запрос
print(sql_query_avg)

# Выполняем запрос и получаем результат
result_avg = average_salary.execute()

# Выводим результат
print(result_avg)




SELECT "t0"."department", AVG("t0"."salary") AS "avg_salary" FROM "employees" AS "t0" GROUP BY 1
    department  avg_salary
0  Engineering     85000.0
1           HR     72500.0
2        Sales     60000.0


#### Объяснение кода

- Мы используем `group_by('department')` для группировки данных по отделам.
- Метод `aggregate(avg_salary=employees_table.salary.mean())` вычисляет среднюю зарплату для каждой группы.
- После выполнения запроса выводится средняя зарплата по каждому отделу.





## 1. Работа с различными источниками данных

### 1.1 Подключение к разным базам данных

Библиотека Ibis предоставляет пользователям возможность осуществлять соединения с различными системами управления базами данных (СУБД). Это делает её универсальным инструментом для анализа данных, так как она поддерживает множество SQL-движков, включая PostgreSQL, MySQL, SQLite, а также распределённые базы данных, такие как Apache Impala и Dask.

Подключение к базе данных осуществляется через создание объекта подключения, в который передаются параметры соединения. В следующем примере мы создадим подключение к PostgreSQL и извлечем данные из таблицы.




In [ ]:
# import ibis

# # Указываем параметры подключения
# postgres_connection = ibis.postgres.connect(
#     host='localhost',
#     database='your_database',
#     user='your_username',
#     password='your_password'
# )

# # Получаем таблицу
# employees_table = postgres_connection.table('employees')




#### Объяснение кода
В этом примере мы используем метод `ibis.postgres.connect()` для создания подключения к базе данных PostgreSQL. После успешного соединения мы получаем доступ к таблице `employees`, что позволяет нам выполнять запросы на основе её данных.

### 1.2 Чтение данных из файлов

В дополнение к работе с СУБД, Ibis также поддерживает загрузку данных из файлов различных форматов, включая CSV и Parquet. Это делает библиотеку удобной для работы с данными, хранящимися в файловой системе.

Пример ниже демонстрирует, как можно загрузить данные из CSV-файла и создать таблицу Ibis для последующего анализа.




In [ ]:
import csv

# Define the employee data
employees = [
    (6, 'Alice', 'HR', 70000),
    (7, 'Bob', 'Engineering', 80000),
    (8, 'Charlie', 'Sales', 60000),
    (9, 'David', 'Engineering', 90000),
    (10, 'Eva', 'HR', 75000)
]

# Define the CSV file name
csv_file = 'employees.csv'

# Write the data to the CSV file
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the header
    writer.writerow(['ID', 'Name', 'Department', 'Salary'])

    # Write the employee data
    writer.writerows(employees)

print(f"CSV file '{csv_file}' created successfully.")

CSV file 'employees.csv' created successfully.


In [ ]:
!pip install ibis-framework

In [ ]:
import ibis
print(ibis.__version__)  # Check the version of Ibis
print(ibis.backends)     # Check available backends

9.2.0
<module 'ibis.backends' from '/usr/local/lib/python3.11/dist-packages/ibis/backends/__init__.py'>


In [ ]:
import pandas as pd
import ibis


# Step 1: Read the CSV file into a Pandas DataFrame
df = pd.read_csv('employees.csv')

# Step 2: Connect to the DataFrame using Ibis
# Use the Pandas backend
connection = ibis.pandas.connect({'employees': df})

# Step 3: Load the DataFrame as an Ibis table
ibis_table = connection.table('employees')

# Step 4: Verify the Ibis table
print(ibis_table)

AttributeError: module 'ibis' has no attribute 'pandas'. 

#### Объяснение кода
В данном случае мы используем библиотеку Pandas для чтения данных из CSV-файла и создания DataFrame. Затем мы инициализируем соединение Ibis с этим DataFrame, что позволяет выполнять аналитические запросы с использованием функциональности Ibis.

## 2. Сложные запросы

### 2.1 Соединение таблиц (Join)

Одной из ключевых операций в анализе данных является объединение таблиц (join), которое позволяет сочетать информацию из различных источников. Ibis предоставляет простую и интуитивно понятную нотацию для выполнения таких операций.

В следующем примере мы рассмотрим, как можно выполнить соединение между таблицей сотрудников и таблицей отделов, чтобы получить информацию о местоположении каждого сотрудника.



In [ ]:
# Создаем таблицу departments
departments_table = postgres_connection.table('departments')

# Выполняем соединение
query = (
    employees_table
    .join(departments_table, employees_table.department == departments_table.name)
    .select(employees_table.name, departments_table.location)
)

# Преобразуем выражение в SQL и выполняем
result = query.execute()
print(result)




NameError: name 'postgres_connection' is not defined


#### Объяснение кода
В этом примере мы используем метод `join()` для соединения таблицы `employees` с таблицей `departments`. Соединение выполняется по условию, что значение столбца `department` в таблице `employees` соответствует имени отдела в таблице `departments`. После этого мы выбираем только необходимые столбцы: имя сотрудника и местоположение отдела. Наконец, запрос выполняется с помощью метода `execute()`, что возвращает результат в виде DataFrame.

### 2.2 Подзапросы и вложенные запросы

Подзапросы, или вложенные запросы, представляют собой мощный инструмент для выполнения более сложных аналитических задач. Ibis поддерживает создание подзапросов, что позволяет пользователям извлекать данные на основе результатов предварительных запросов.

Рассмотрим пример, в котором мы сначала создаем подзапрос для выбора сотрудников с высокой зарплатой, а затем используем этот подзапрос в основном запросе.




In [ ]:
# Создаем подзапрос для получения сотрудников с высокой зарплатой
high_salary_subquery = employees_table[employees_table.salary > 80000]

# Основной запрос, который использует подзапрос
query = employees_table[employees_table.id.isin(high_salary_subquery.id)]

# Выполняем запрос
result = query.execute()
print(result)




   id   name   department   salary
0   4  David  Engineering  90000.0


#### Объяснение кода
В этом примере мы создаем подзапрос `high_salary_subquery`, который выбирает сотрудников с зарплатой выше 80000. Затем основной запрос использует метод `isin()` для фильтрации сотрудников по идентификаторам, которые находятся в подзапросе. В результате мы получаем всех сотрудников с высокой зарплатой.

## 3. Анализ временных рядов

### 3.1 Работа с датами и временем

Анализ временных рядов — это важная область в аналитике данных, которая требует обработки и анализа временных данных. Ibis предоставляет обширные возможности для работы с датами, позволяя пользователям агрегировать данные по временным интервалам и выполнять различные временные операции.

В следующем примере мы рассмотрим, как агрегировать данные о продажах по месяцам, чтобы получить общую сумму продаж за каждый месяц.




In [ ]:
sales_table = ibis_connection.table('sales')

# Группируем данные по месяцу и считаем сумму продаж
monthly_sales = (
    sales_table
    .group_by([sales_table.sale_date.dt.year(), sales_table.sale_date.dt.month()])
    .aggregate(total_sales=sales_table.amount.sum())
)

# Выполняем запрос
result = monthly_sales.execute()
print(result)




IbisError: Table not found: 'sales'

#### Объяснение кода
В этом примере мы используем методы `dt.year()` и `dt.month()` для извлечения года и месяца из столбца с временной меткой `sale_date`. Затем с помощью метода `group_by()` мы группируем данные по этим временным меткам и используем `aggregate()` для подсчета общей суммы продаж за каждый месяц.

## 4. Обработка отсутствующих данных

### 4.1 Работа с пропущенными значениями

Пропущенные значения в данных могут существенно повлиять на результаты анализа. Ibis предоставляет пользователям инструменты для обработки отсутствующих данных, включая возможность фильтрации и замены.

В следующем примере мы удалим записи, в которых отсутствуют значения зарплаты, чтобы обеспечить корректность анализа.




In [ ]:
# Фильтруем записи с отсутствующими значениями зарплаты
cleaned_data = employees_table[employees_table.salary.notnull()]

# Выполняем запрос
result = cleaned_data.execute()
print(result)




   id     name   department   salary
0   1    Alice           HR  70000.0
1   2      Bob  Engineering  80000.0
2   3  Charlie        Sales  60000.0
3   4    David  Engineering  90000.0
4   5      Eva           HR  75000.0


#### Объяснение кода
В этом коде мы используем метод `notnull()` для фильтрации записей, где значение в столбце `salary` отсутствует. Это позволяет нам создать новую таблицу `cleaned_data`, содержащую только те записи, которые имеют действительные значения зарплаты. После этого мы выполняем запрос, получая очищенные данные.

## 5. Оптимизация запросов

### 5.1 Профилирование запросов

Оптимизация производительности запросов является критически важной задачей при работе с большими наборами данных. Ibis предоставляет инструменты для профилирования запросов, позволяя пользователям анализировать время выполнения и выявлять узкие места.

В следующем примере мы воспользуемся встроенной функцией Jupyter для измерения времени выполнения запроса.




In [ ]:
# Анализируем производительность запроса
query = employees_table[employees_table.salary > 70000]
# Сохраняем время выполнения запроса
%timeit query.execute()




5.42 ms ± 957 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Объяснение кода
В данном примере мы создаем запрос, который выбирает сотрудников с зарплатой выше 70000. С помощью команды Jupyter `%timeit` мы измеряем время выполнения этого запроса. Это помогает определить, насколько эффективно работает запрос, и выявить необходимость в оптимизации.

## 6. Интеграция с Pandas

### 6.1 Конвертация между Ibis и Pandas

Ibis позволяет удобно интегрироваться с библиотекой Pandas, что упрощает процесс анализа данных и визуализации. Пользователи могут легко конвертировать данные между Ibis и Pandas, что позволяет использовать мощные инструменты анализа из обеих библиотек.

В следующем примере мы рассмотрим, как конвертировать таблицу Ibis в DataFrame Pandas и обратно.




In [ ]:
# Конвертируем таблицу Ibis в DataFrame Pandas
df_result = employees_table.execute()

# Преобразуем обратно в Ibis
ibis_table_from_df = ibis.pandas.connect({'employees': df_result}).table('employees')




ImportError: Failed to import the pandas backend due to missing dependencies.

You can pip or conda install the pandas backend as follows:

  python -m pip install -U "ibis-framework[pandas]"  # pip install
  conda install -c conda-forge ibis-pandas           # or conda install

#### Объяснение кода
Сначала мы выполняем запрос к таблице Ibis и сохраняем результат в DataFrame Pandas. Затем мы создаем новое подключение Ibis на основе этого DataFrame, что позволяет использовать все преимущества Ibis для дальнейшего анализа.

## 7. Группировка и агрегация данных

### 7.1 Различные агрегатные функции

Агрегация данных является одной из основных операций при анализе больших наборов данных. Ibis поддерживает множество встроенных агрегатных функций, включая вычисление среднего, суммы, минимума и максимума.

Рассмотрим пример, где мы вычисляем минимальную, максимальную и среднюю зарплату сотрудников по отделам.




In [ ]:
# Получаем минимальную, максимальную и среднюю зарплату по отделам
salary_stats = (
    employees_table.group_by('department')
    .aggregate(
        min_salary=employees_table.salary.min(),
        max_salary=employees_table.salary.max(),
        avg_salary=employees_table.salary.mean()
    )
)

# Выполняем запрос
result = salary_stats.execute()
print(result)




    department  min_salary  max_salary  avg_salary
0  Engineering     80000.0     90000.0     85000.0
1           HR     70000.0     75000.0     72500.0
2        Sales     60000.0     60000.0     60000.0


#### Объяснение кода
В данном коде мы используем метод `group_by()` для группировки данных по отделам. Затем применяем агрегатные функции `min()`, `max()` и `mean()` для вычисления минимальной, максимальной и средней зарплаты в каждом отделе.


## Заключение

Ibis — мощный инструмент для работы с данными, позволяющий пользователям легко и интуитивно создавать запросы, используя высокоуровневый синтаксис Python. Библиотека поддерживает различные источники данных и предоставляет возможности агрегации, фильтрации и группировки, что делает её полезной для аналитиков и разработчиков.




### Вопросы для самопроверки

1. Что такое Ibis и для чего она используется?

2. Как установить библиотеку Ibis с помощью pip?

3. Какие основные источники данных поддерживает Ibis?

4. Как создать выражение в Ibis для фильтрации данных?

5. Что такое Ibis Expressions и как они работают?

6. Как подключиться к базе данных SQLite с помощью Ibis?

7. Как выполнить агрегацию данных по столбцу в Ibis?

8. Что делает метод `compile()` в Ibis?

9. Как использовать функции окон в Ibis для анализа данных?

10. Какие методы Ibis можно использовать для работы с пропущенными значениями?

11. Как выполнять соединение (join) между двумя таблицами в Ibis?

12. Что такое подзапросы и как их реализовать в Ibis?

13. Как можно работать с временными рядами в Ibis?

14. Что такое `execute()` и как он используется в Ibis?

15. Как конвертировать таблицу Ibis в DataFrame Pandas?

16. Как выполнить группировку данных по нескольким столбцам в Ibis?

17. Что такое агрегация и какие функции Ibis поддерживают её?

18. Как можно работать с файлами формата CSV в Ibis?

19. Как оптимизировать запросы в Ibis и что для этого нужно делать?

20. Как сохранить результаты запроса в новый CSV-файл с помощью Ibis?



## Задачи для самостоятеьной работы

1. **Установка Ibis**: Установите библиотеку Ibis и проверьте её версию.

2. **Создание базы данных**: Создайте SQLite базу данных с таблицей `products`, содержащей поля `id`, `name`, `category`, `price`.

3. **Заполнение таблицы**: Добавьте в таблицу `products` как минимум 10 записей с различными категориями и ценами.

4. **Подключение к базе данных**: Подключитесь к созданной базе данных с помощью Ibis.

5. **Выборка данных**: Напишите запрос, который выберет все продукты с ценой выше 50.

6. **Группировка и агрегация**: Создайте запрос, который подсчитает количество продуктов в каждой категории.

7. **Фильтрация данных**: Напишите запрос, который выберет все продукты из категории "Electronics" с ценой ниже 100.

8. **Обновление данных**: Обновите цену для продукта с `id=1` на 75.

9. **Удаление данных**: Удалите все продукты из категории "Furniture".

10. **Подключение к PostgreSQL**: Подключитесь к базе данных PostgreSQL и извлеките данные из таблицы `employees`.

11. **Чтение данных из CSV**: Загрузите данные из CSV-файла в Ibis и создайте таблицу.

12. **Соединение таблиц**: Выполните соединение между таблицами `employees` и `departments` по полю `department_id`.

13. **Подзапросы**: Напишите запрос, который использует подзапрос для фильтрации сотрудников с зарплатой выше средней.

14. **Анализ временных рядов**: Создайте таблицу с данными о продажах и выполните группировку по месяцам, подсчитав общую сумму продаж.

15. **Работа с пропущенными значениями**: Фильтруйте записи с отсутствующими значениями в столбце `salary`.

16. **Оптимизация запроса**: Измерьте время выполнения запроса на выборку всех сотрудников с зарплатой выше 70000.

17. **Интеграция с Pandas**: Конвертируйте таблицу Ibis в DataFrame Pandas и выполните базовый анализ данных.

18. **Создание сложных запросов**: Создайте запрос, который объединяет данные из нескольких таблиц и выполняет агрегацию.

19. **Работа с датами**: Создайте запрос, который выбирает записи за последний месяц.

20. **Анализ категорий**: Подсчитайте среднюю цену продуктов по категориям.

21. **Функции окон**: Используйте функции окон для расчета рангов продуктов по цене.

22. **Фильтрация по нескольким условиям**: Напишите запрос, который выбирает продукты, удовлетворяющие нескольким условиям (например, цена и категория).

23. **Сохранение результатов**: Сохраните результаты запроса в новый CSV-файл.

24. **Создание индексов**: Создайте индекс для столбца `category` в таблице `products`.

25. **Работа с Parquet**: Загрузите данные из Parquet-файла и выполните базовый анализ.

26. **Использование агрегатных функций**: Используйте функции `min()`, `max()`, и `mean()` для анализа зарплат сотрудников.

27. **Создание представлений**: Создайте представление для часто используемого запроса.

28. **Форматирование данных**: Преобразуйте значения в столбце `price` в формат с двумя знаками после запятой.

29. **Визуализация данных**: Используйте Pandas для визуализации результатов запросов с помощью Matplotlib или Seaborn.

30. **Документация**: Напишите краткое руководство по использованию Ibis с примерами кода.
